In [3]:
import numpy as np
import pandas as pd

In [4]:
pd.set_option('display.max_columns', None)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 분석 목표

- 8월 총 대여이력에서 고장신고 접수된 대여이력과 고장이 없었던 반납이력을
- 고장여부 (0=정상, 1=고장)으로 표시
- 고장여부 판단은 반납 후 고장신고 접수 여부를 통해 확인
- 즉, 고장신고 전 마지막으로 반납된 시간을 찾아, 고장으로 표시

# 사용 데이터

In [ ]:
- 1. 8월대여이력전처리후
- 2. 8월 고장 신고 전 마지막 사용 기록

## 데이터 업로드

In [5]:
rental_data_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/스프린트미션/프로젝트1/data/8월대여이력전처리후.csv'
rental_df = pd.read_csv(rental_data_path, encoding='utf-8-sig')

In [6]:
rental_df.head(2)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명
0,SPB-69706,2024-08-01 00:00:24,4600.0,영등포구 총괄 우체국,2024-08-01 00:02:54,288,쌍용예가(구청별관)
1,SPB-43038,2024-08-01 00:00:22,563.0,성동세무서 건너편,2024-08-01 00:03:17,4383,푸조비즈타워 앞


In [7]:
broke_data_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/스프린트미션/프로젝트1/data/8월 고장신고내역 + 자치구.csv'
broke_df = pd.read_csv(broke_data_path, encoding='utf-8-sig')

In [8]:
broke_df.sort_values(by='등록일시')

,자전거번호,등록일시,구분,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,자치구,시간차
2202,SPB-38515,2024-08-01 00:58:32,페달,2024-08-01 00:54:45,816.0,신용산역 6번출구 앞,2024-08-01 00:56:54,845.0,용산 선인상가,용산구,98.0
5810,SPB-50070,2024-08-01 01:21:43,기타,2024-08-01 01:14:34,361.0,동묘앞역 1번출구 뒤,2024-08-01 01:19:15,431.0,청계천 영도교,중구,148.0
113,SPB-30411,2024-08-01 02:04:59,페달,2024-08-01 02:02:11,1015.0,샛마을 근린공원,2024-08-01 02:04:30,3698.0,명일역 3번출구 옆,강동구,29.0
5974,SPB-50577,2024-08-01 02:17:57,체인,2024-08-01 01:10:07,4791.0,명동역9번 출구,2024-08-01 02:17:27,2002.0,노들역 1번출구,동작구,30.0
2115,SPB-38220,2024-08-01 02:28:47,체인,2024-08-01 02:20:08,906.0,연신내역 5번출구,2024-08-01 02:26:23,932.0,예일여중,은평구,144.0
...,...,...,...,...,...,...,...,...,...,...,...
10874,SPB-65506,2024-08-31 23:49:03,체인,2024-08-31 23:44:54,1357.0,북한산보국문역,2024-08-31 23:47:52,1357.0,북한산보국문역,성북구,71.0
6767,SPB-52993,2024-08-31 23:54:03,타이어,2024-08-31 23:48:47,3807.0,관악우체국 교차로,2024-08-31 23:53:33,2189.0,봉천역 4번출구,관악구,30.0
10363,SPB-64175,2024-08-31 23:54:31,기타,2024-08-31 23:44:15,641.0,용두역 4번출구,2024-08-31 23:52:57,642.0,신답역 사거리,동대문구,94.0
12662,SPB-69994,2024-08-31 23:57:30,체인,2024-08-31 23:15:49,502.0,자양(뚝섬한강공원)역 1번출구 앞,2024-08-31 23:56:51,3600.0,사근빗물펌프장 건너편,성동구,39.0


# 데이터 구조 확인

In [9]:
rental_df.head(2)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명
0,SPB-69706,2024-08-01 00:00:24,4600.0,영등포구 총괄 우체국,2024-08-01 00:02:54,288,쌍용예가(구청별관)
1,SPB-43038,2024-08-01 00:00:22,563.0,성동세무서 건너편,2024-08-01 00:03:17,4383,푸조비즈타워 앞


In [10]:
broke_df.head(2)

,자전거번호,등록일시,구분,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,자치구,시간차
0,SPB-30006,2024-08-13 18:02:24,체인,2024-08-13 11:27:44,184.0,7번가피자 서교망원점,2024-08-13 11:38:34,113.0,홍대입구역 2번출구 앞,마포구,23030.0
1,SPB-30014,2024-08-29 08:21:44,기타,2024-08-29 08:08:40,179.0,가좌역 4번출구 앞,2024-08-29 08:20:37,195.0,모래내고가차도,서대문구,67.0


In [11]:
rental_df.shape, broke_df.shape

((3997344, 7), (13058, 11))

In [12]:
rental_df.drop_duplicates().shape

(3997029, 7)

# 데이터 합치기

### 전략

In [13]:
rental_df.columns

Index(['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '반납일시', '반납대여소번호', '반납대여소명'], dtype='object')

In [14]:
broke_df.columns

Index(['자전거번호', '등록일시', '구분', '대여일시', '대여 대여소번호', '대여 대여소명', '반납일시', '반납대여소번호',
       '반납대여소명', '자치구', '시간차'],
      dtype='object')

- 대여이력: 자전거번호, 반납일시, 반납대여소명
- 고장신고: 자전거번호, 반납일시, 반납대여소명
-> 모두 일치하는 데이터에 ['고장여부'] = 1(고장)
-> 일치하지 않는 데이터에 ['고장여부'] = 0(정상)
-> 0과 1일 비율을 도넛차트로 시각화가능

In [15]:
rental = rental_df.copy()

In [16]:
rental.tail(2)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명
3997342,SPB-50174,2024-08-31 23:40:15,2730.0,마곡메가박스 앞,2024-09-01 03:24:43,1113,서남환경공원 버스정류장
3997343,SPB-41904,2024-08-31 23:46:05,3009.0,홍대입구역 사거리,2024-09-01 04:57:59,245,삼성생명 당산사옥 앞


In [17]:
# 날짜데이터 확인
rental['반납일시'].dtypes, broke_df['반납일시'].dtypes

(dtype('O'), dtype('O'))

In [18]:
rental['반납일시'].unique()

array(['2024-08-01 00:02:54', '2024-08-01 00:03:17',
       '2024-08-01 00:03:18', ..., '2024-09-01 03:23:53',
       '2024-09-01 03:24:43', '2024-09-01 04:57:59'], dtype=object)

In [19]:
# 날짜데이터 변환
rental['반납일시'] = pd.to_datetime(rental['반납일시'])
broke_df['반납일시'] = pd.to_datetime(broke_df['반납일시'])

### 고장여부 표시하기

- 대여이력 데이터 3,397,344 개
- 고장신고 데이터 13,058 개
- 대여이력=고장신고 데이터 12,975 (83개의 데이터 / 73대의 자전거에 대한 정보가 없음)
- '자치구'를 찾기 위한 "8월고장분석"에서는 71대의 자전거 정보가 없었음 -> 2대 더 늘어났지만 큰 수치 아님으로 넘어가기

#### 고장여부 컬럼 생성

In [20]:
# 고장여부 컬럼 생성하여 기본값으로 0 부여
rental['고장여부'] = 0

In [21]:
rental.tail(3)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,고장여부
3997341,SPB-58360,2024-08-31 23:41:36,2730.0,마곡메가박스 앞,2024-09-01 03:23:53,1113,서남환경공원 버스정류장,0
3997342,SPB-50174,2024-08-31 23:40:15,2730.0,마곡메가박스 앞,2024-09-01 03:24:43,1113,서남환경공원 버스정류장,0
3997343,SPB-41904,2024-08-31 23:46:05,3009.0,홍대입구역 사거리,2024-09-01 04:57:59,245,삼성생명 당산사옥 앞,0


#### 고장 데이터와 일치하는 행의 고장여부를 1로 변경하기

In [ ]:
# 두 파일의 인덱스를 '자전거번호' '반납일시'로 설정하기
# 인덱스 번호가 일치하는 인덱스 찾아내기

In [22]:
고장신고_8월 = rental[
    rental.set_index(['자전거번호', '반납일시']).index.isin(broke_df.set_index(['자전거번호', '반납일시']).index)
]

In [23]:
rental.loc[
    rental.set_index(['자전거번호', '반납일시']).index.isin(broke_df.set_index(['자전거번호', '반납일시']).index), '고장여부'
] = 1

In [24]:
# 고장여부가 잘 들어갔는지 확인
고장신고_8월.shape, rental['고장여부'].sum()

((12979, 8), 12979)

# 최종데이터

In [25]:
rental.head(10)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,고장여부
0,SPB-69706,2024-08-01 00:00:24,4600.0,영등포구 총괄 우체국,2024-08-01 00:02:54,288,쌍용예가(구청별관),0
1,SPB-43038,2024-08-01 00:00:22,563.0,성동세무서 건너편,2024-08-01 00:03:17,4383,푸조비즈타워 앞,0
2,SPB-81186,2024-08-01 00:01:37,223.0,진주아파트상가 앞,2024-08-01 00:03:18,4563,여의도 은하아파트,0
3,SPB-56794,2024-08-01 00:00:05,1044.0,굽은다리역,2024-08-01 00:03:22,1073,SSTS 몰 앞,0
4,SPB-69344,2024-08-01 00:00:09,3757.0,신방화역 6번출구,2024-08-01 00:03:32,1112,마곡엠밸리4단지 정문,0
5,SPB-62095,2024-08-01 00:00:22,231.0,남부고용노동지청 남측,2024-08-01 00:03:48,5866,포레나 당산,0
6,SPB-64813,2024-08-01 00:00:12,562.0,군자지하보도 앞,2024-08-01 00:03:48,3500,군자역2번출구,0
7,SPB-32071,2024-08-01 00:02:00,1667.0,중계중학교,2024-08-01 00:04:08,1659,중계동 을지중학교,0
8,SPB-52354,2024-08-01 00:00:03,571.0,세종대학교 대양AI센터,2024-08-01 00:04:10,3509,세종사이버대학교,0
9,SPB-43669,2024-08-01 00:01:20,976.0,진관고등학교앞,2024-08-01 00:04:16,903,은평뉴타운 아이파크,0


In [26]:
rental_final = rental[['자전거번호', '대여일시', '반납일시', '반납대여소번호', '고장여부']]

In [27]:
rental.shape, rental_final.shape

((3997344, 8), (3997344, 5))

In [ ]:
# 8월 대여자전거 고장여부 추가 파일
#rental_final.to_csv('8월대여_고장여부_추가', index=False, encoding='utf-8-sig')

In [ ]:
# path = '/content/8월대여_고장여부_추가'
# 확인 = pd.read_csv(path, encoding='utf-8-sig')

In [ ]:
#확인.shape

### 자치구 추가하기

In [28]:
station_info_data_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/스프린트미션/프로젝트1/data/raw_data/공공자전거 대여소 정보(24.12월 기준).txt'
station_info = pd.read_csv(station_info_data_path, delimiter=",", encoding="utf-8")

In [29]:
station_info.head(2)

,대여소번호,대여소명,자치구,상세주소,위도,경도,설치시기,LCD,QR,운영방식
0,301,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,2015-10-07,20.0,20.0,QR
1,302,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,2015-10-07,12.0,12.0,QR


In [30]:
rental['반납대여소번호'].dtypes, station_info['대여소번호'].dtypes

(dtype('int64'), dtype('int64'))

In [31]:
rental.head(1)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,고장여부
0,SPB-69706,2024-08-01 00:00:24,4600.0,영등포구 총괄 우체국,2024-08-01 00:02:54,288,쌍용예가(구청별관),0


In [32]:
rental_merged = rental.merge(station_info[['대여소번호', '자치구']], left_on='반납대여소번호', right_on='대여소번호', how='left')

In [34]:
rental_merged

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,고장여부,대여소번호,자치구
0,SPB-69706,2024-08-01 00:00:24,4600.0,영등포구 총괄 우체국,2024-08-01 00:02:54,288,쌍용예가(구청별관),0,288.0,영등포구
1,SPB-43038,2024-08-01 00:00:22,563.0,성동세무서 건너편,2024-08-01 00:03:17,4383,푸조비즈타워 앞,0,4383.0,성동구
2,SPB-81186,2024-08-01 00:01:37,223.0,진주아파트상가 앞,2024-08-01 00:03:18,4563,여의도 은하아파트,0,4563.0,영등포구
3,SPB-56794,2024-08-01 00:00:05,1044.0,굽은다리역,2024-08-01 00:03:22,1073,SSTS 몰 앞,0,1073.0,강동구
4,SPB-69344,2024-08-01 00:00:09,3757.0,신방화역 6번출구,2024-08-01 00:03:32,1112,마곡엠밸리4단지 정문,0,1112.0,강서구
...,...,...,...,...,...,...,...,...,...,...
3997339,SPB-64092,2024-08-31 20:35:32,3780.0,등촌서광아파트,2024-09-01 03:09:03,1122,황금내근린공원,0,1122.0,강서구
3997340,SPB-45143,2024-08-31 23:56:12,2015.0,신대방삼거리역 6번출구쪽,2024-09-01 03:23:20,853,용산역 맞은편,0,853.0,용산구
3997341,SPB-58360,2024-08-31 23:41:36,2730.0,마곡메가박스 앞,2024-09-01 03:23:53,1113,서남환경공원 버스정류장,0,1113.0,강서구
3997342,SPB-50174,2024-08-31 23:40:15,2730.0,마곡메가박스 앞,2024-09-01 03:24:43,1113,서남환경공원 버스정류장,0,1113.0,강서구


In [35]:
rental_merged_2 = rental_merged.drop(columns=['대여일시','대여 대여소번호', '대여 대여소명', '대여소번호'])

In [36]:
rental_merged_2.head(1)

,자전거번호,반납일시,반납대여소번호,반납대여소명,고장여부,자치구
0,SPB-69706,2024-08-01 00:02:54,288,쌍용예가(구청별관),0,영등포구


In [37]:
rental_merged_2.shape

(3997344, 6)

In [ ]:
# 8월 대여자전거 고장여부 + 자치구 추가 파일
# rental_merged_2.to_csv('8월대여_고장여부_자치구_추가', index=False, encoding='utf-8-sig')